## Task 1

In [2]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        self.constraints = self.generate_constraints()

    def generate_constraints(self):
        """
        Add row column constraints and subgrid constraints
        """
        constraints = []

        for row in range(9):
            constraints.append([(row, col) for col in range(9)])

        for col in range(9):
            constraints.append([(row, col) for row in range(9)])

        for row_start in range(0, 9, 3):
            for col_start in range(0, 9, 3):
                subgrid = [
                    (row_start + i, col_start + j)
                    for i in range(3)
                    for j in range(3)
                ]
                constraints.append(subgrid)

        return constraints

    def is_valid(self, var, value):
        """
        Check if assigning 'value' to 'var' satisfies all constraints.
        """
        row, col = var

        # Check row uniqueness
        for c in range(9):
            if c != col and self.grid[row][c] == value:
                return False

        # Check column uniqueness
        for r in range(9):
            if r != row and self.grid[r][col] == value:
                return False

        # Check subgrid uniqueness
        subgrid_row_start = (row // 3) * 3
        subgrid_col_start = (col // 3) * 3
        for i in range(3):
            for j in range(3):
                r = subgrid_row_start + i
                c = subgrid_col_start + j
                if (r != row or c != col) and self.grid[r][c] == value:
                    return False

        return True

    def forward_checking(self, var, value):
        """
        Remove 'value' from the domain of neighbors of 'var'.
        Return list of affected variables to restore later.
        """
        affected = []
        row, col = var

        # Remove value from row domains
        for c in range(9):
            if c != col and value in self.domains[(row, c)]:
                self.domains[(row, c)].remove(value)
                affected.append((row, c))
                if not self.domains[(row, c)]:  # If domain is empty
                    self.restore_domains(affected, value)
                    return None

        # Remove value from column domains
        for r in range(9):
            if r != row and value in self.domains[(r, col)]:
                self.domains[(r, col)].remove(value)
                affected.append((r, col))
                if not self.domains[(r, col)]:
                    self.restore_domains(affected, value)
                    return None

        # Remove value from subgrid domains
        subgrid_row_start = (row // 3) * 3
        subgrid_col_start = (col // 3) * 3
        for i in range(3):
            for j in range(3):
                r = subgrid_row_start + i
                c = subgrid_col_start + j
                if (r != row or c != col) and value in self.domains[(r, c)]:
                    self.domains[(r, c)].remove(value)
                    affected.append((r, c))
                    if not self.domains[(r, c)]:
                        self.restore_domains(affected, value)
                        return None

        return affected

    def restore_domains(self, affected, value):
        """
        Restore removed values after backtracking.
        """
        for var in affected:
            self.domains[var].add(value)

    def select_unassigned_variable(self):
        """
        Select the next variable to assign using the MRV heuristic.
        """
        unassigned_vars = [var for var in self.variables if self.grid[var[0]][var[1]] == 0]
        if not unassigned_vars:
            return None
        return min(unassigned_vars, key=lambda var: len(self.domains[var]))

    def backtrack(self):
        """
        Solve the puzzle using backtracking + MRV + forward checking.
        """
        var = self.select_unassigned_variable()
        if var is None:
            return True  # All variables are assigned

        row, col = var
        for value in sorted(self.domains[var]):  # Sorting to try smaller values first
            if self.is_valid(var, value):
                # Assign value to the variable
                self.grid[row][col] = value

                # Forward checking
                affected = self.forward_checking(var, value)
                if affected is None:  # If forward checking detected an issue
                    self.grid[row][col] = 0
                    continue

                # Recursive call
                if self.backtrack():
                    return True

                # Backtrack: undo assignment and restore domains
                self.grid[row][col] = 0
                self.restore_domains(affected, value)

        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None


In [3]:

# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
